In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('X_train.csv')

# UserName

In [3]:
t = {}
i = 1

for user in df['userName'].unique():
    t[user] = i
    i += 1
    
for user in t:
    df.set_value(df['userName'] == user, 'userName', t[user])
    
df['userName'] = pd.to_numeric(df['userName'])

# Comments

In [4]:
import itertools
import operator
from collections import Counter

In [5]:
bad = df[df['reting'] <= 3.5]['comment']
splits = list(map(lambda x: x.split(' '), bad))
arr = list(itertools.chain.from_iterable(splits))
bad = dict(Counter(arr))
sorted_bad = sorted(bad.items(), key=operator.itemgetter(1), reverse=True)
sorted_bad[: 20]

[('не', 4892),
 ('и', 4425),
 ('', 3695),
 ('в', 3279),
 ('на', 2758),
 ('-', 1722),
 ('что', 1616),
 ('с', 1433),
 ('но', 1050),
 ('а', 918),
 ('как', 808),
 ('очень', 798),
 ('по', 767),
 ('это', 766),
 ('для', 765),
 ('при', 721),
 ('только', 638),
 ('через', 623),
 ('все', 607),
 ('так', 584)]

In [6]:
good = df[df['reting'] > 3.5]['comment']
splits = list(map(lambda x: x.split(' '), good))
arr = list(itertools.chain.from_iterable(splits))
good = dict(Counter(arr))
sorted_good = sorted(good.items(), key=operator.itemgetter(1), reverse=True)
sorted_good[: 20]

[('и', 16197),
 ('не', 13812),
 ('', 13092),
 ('в', 11128),
 ('на', 8793),
 ('-', 5509),
 ('с', 5258),
 ('что', 5182),
 ('очень', 4425),
 ('но', 3401),
 ('для', 3360),
 ('как', 2720),
 ('все', 2676),
 ('по', 2567),
 ('это', 2524),
 ('а', 2331),
 ('у', 2173),
 ('за', 2157),
 ('я', 1916),
 ('так', 1771)]

# Properties

In [7]:
for i in range(0, df.shape[0]):
    props = df['property'][i].split(', ')
    q = map(lambda x: x.split(': '), props)
    props_new = list(q)
    
    res = []

    for j in range(0, len(props_new)):
        res.append(props_new[j][0].split('{')[1])
    
    df.set_value(i, 'property', res)

## Select properties

In [8]:
good = df[df['reting'] > 3.5]['property']
arr = list(itertools.chain.from_iterable(good))
good = dict(Counter(arr))

props = pd.DataFrame()
props['props'] = good.keys()
props['count'] = good.values()

props_good = list(props[(props['count'] < 500) & (props['count'] > 300)]['props'].astype(int))
#props_good = list(props[(props['count'] > 1000)]['props'].astype(int))
len(props_good)

79

In [9]:
bad = df[df['reting'] <= 3.5]['property']
arr = list(itertools.chain.from_iterable(bad))
bad = dict(Counter(arr))

props = pd.DataFrame()
props['props'] = bad.keys()
props['count'] = bad.values()

props_bad = list(props[(props['count'] < 900) & (props['count'] > 200)]['props'].astype(int))
#props_bad = list(props[(props['count'] > 1000)]['props'].astype(int))
len(props_bad)

55

In [10]:
props_good_new = set(props_good) - set(props_bad)
props_bad_new = set(props_bad) - set(props_good)

props_good = props_good_new
props_bad = props_bad_new

print(len(props_good))
print(len(props_bad))

77
53


In [11]:
for j in range(0, df.shape[0]):
    foo = df['property'][j]

    count_good = 0
    count_bad = 0

    for i in range(0, len(foo)):
        if int(foo[i]) in props_good:
            count_good += 1

        if int(foo[i]) in props_bad:
            count_bad += 1
    
#     if count_good > 3:
#         df.set_value(j, 'is_good_probs', count_good)#True)
#     else:
#         df.set_value(j, 'is_good_probs', False)
        
#     if count_bad > 3:
#         df.set_value(j, 'is_bad_probs', count_bad)#True)
#     else:
#         df.set_value(j, 'is_good_probs', False)
        
    df.set_value(j, 'is_good_probs', int(count_good))
    df.set_value(j, 'is_bad_probs', int(count_bad))

In [12]:
df_prop = df[['reting', 'is_good_probs', 'is_bad_probs']].astype(int)
#df_prop

In [13]:
limit = 10

In [14]:
# много свойств в плохом и плохой рейтинг

df[(df['is_bad_probs'] > limit) & (df['reting'] < 4)].shape[0]

399

In [15]:
# много свойств в плохом и хороший рейтинг

df[(df['is_bad_probs'] > limit) & (df['reting'] > 3)].shape[0]

1561

In [16]:
# много свойств в хорошем и хороший рейтинг

df[(df['is_good_probs'] > limit) & (df['reting'] > 3)].shape[0]

705

In [17]:
# много свойств в хорошем и плохой рейтинг

df[(df['is_good_probs'] > limit) & (df['reting'] < 4)].shape[0]

205

In [18]:
for i in range(0, df.shape[0]):
    if df['is_bad_probs'][i] > limit:
        df['is_good_bool'] = True
    else:
        df['is_good_bool'] = False

# Comments

In [19]:
for i in range(0, df.shape[0]):
    if len(df['comment'][i].split(' ')) > 6:
        df['long'] = True
    else:
        df['long'] = False

In [20]:
from math import isnan

In [21]:
not isnan(df['commentNegative'][i])

False

In [22]:
not isnan(df['commentNegative'][i])

False

# Prediction

In [25]:
y = df['reting'].astype(int)
X = df.drop(['categoryLevel1Id', 'reting', 'date', 'commentNegative', 'commentPositive', 'property',
             'comment', 'is_good_probs', 'is_bad_probs'], axis=1)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
X.head()

,sku,categoryLevel2Id,brandId,userName,is_good_bool,long
0,20005023,4010201,826,1,False,True
1,20020647,4030101,1425,2,False,True
2,20020701,4010401,124,3,False,True
3,30012256,2030301,93,4,False,True
4,30011341,2050201,656,5,False,True


In [28]:
boost = xgb.XGBClassifier(max_depth=15, n_estimators=200)
boost.fit(X_train, y_train)
y_pred = boost.predict(X_test)
print(classification_report(list(y_test), list(y_pred)))

             precision    recall  f1-score   support

          1       0.27      0.14      0.18       503
          2       0.18      0.07      0.10       289
          3       0.15      0.04      0.06       439
          4       0.24      0.14      0.18       878
          5       0.62      0.85      0.72      3035

avg / total       0.46      0.54      0.48      5144



In [29]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          1       0.18      0.19      0.19       503
          2       0.10      0.11      0.10       289
          3       0.12      0.13      0.12       439
          4       0.21      0.22      0.21       878
          5       0.64      0.62      0.63      3035

avg / total       0.45      0.44      0.44      5144



In [30]:
tree = ExtraTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          1       0.21      0.22      0.21       503
          2       0.10      0.10      0.10       289
          3       0.11      0.10      0.10       439
          4       0.21      0.21      0.21       878
          5       0.64      0.64      0.64      3035

avg / total       0.45      0.45      0.45      5144



# Лян

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [46]:
y = df['reting'].astype(int)
X = df.drop(['categoryLevel1Id', 'reting', 'date', 'commentNegative', 'commentPositive', 'property',
             'is_good_probs', 'is_bad_probs'], axis=1)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [35]:
tfidf = TfidfTransformer()
cv = CountVectorizer(max_features=100)

ft = tfidf.fit_transform(cv.fit_transform(X_train.comment))
t_ft = tfidf.fit_transform(cv.fit_transform(X_test.comment))

In [39]:
X_train = ft.toarray()
X_test = t_ft.toarray()

In [40]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [48]:
%%time
param = {'silent':1, 'objective':'multi:softmax' }
param['nthread'] = 8
param['eval_metric'] = 'merror'
param['eta'] = 0.075
param['max_depth'] = 5
param['silent'] = 1
param['num_class'] = 7
param['n_estimators'] = 300
param['early_stopping_rounds'] = 30
param['learning_rates'] = 0.2

watchlist = [ (dtrain,'train'), (dtest, 'test') ]

bst = xgb.train(param, dtrain, 500, watchlist, verbose_eval=25)

[0]	train-merror:0.393086	test-merror:0.413103
[25]	train-merror:0.384851	test-merror:0.408243
[50]	train-merror:0.372115	test-merror:0.409409
[75]	train-merror:0.356794	test-merror:0.411936
[100]	train-merror:0.339845	test-merror:0.412131
[125]	train-merror:0.325768	test-merror:0.413297
[150]	train-merror:0.31399	test-merror:0.416213
[175]	train-merror:0.303936	test-merror:0.419323
[200]	train-merror:0.296945	test-merror:0.421851
[225]	train-merror:0.289668	test-merror:0.423212
[250]	train-merror:0.280379	test-merror:0.426905
[275]	train-merror:0.271953	test-merror:0.426905
[300]	train-merror:0.264771	test-merror:0.42846
[325]	train-merror:0.257972	test-merror:0.429627
[350]	train-merror:0.251556	test-merror:0.43021
[375]	train-merror:0.244278	test-merror:0.431571
[400]	train-merror:0.238341	test-merror:0.431765
[425]	train-merror:0.230681	test-merror:0.431571
[450]	train-merror:0.225989	test-merror:0.432737
[475]	train-merror:0.220435	test-merror:0.433904
CPU times: user 2min 50s, sy